# 📘 02_merge_weather.ipynb
## 🧩 Purpose
This notebook merges the public bike rental data with the daily weather data of Seoul, in order to enable analysis and modeling based on environmental factors.

## 📥 Input Datasets
- data/processed/merged_bike_data.csv: Preprocessed bike rental logs (June to August 2023)
- data/raw/seoul_weather.csv: Daily weather data (temperature, precipitation, wind, humidity)

## 🎯 Goal
- Drop unnecessary columns from merged_bike_data
- Convert both datasets to a common date format
- Merge them using date as the key
- Save the final merged dataset to data/processed/bike_with_weather.csv

## ✅ Output
- bike_with_weather.csv: Merged dataset combining bike rentals and weather conditions

In [1]:
import pandas as pd

In [2]:
# Bike data set(preprocessed) 
bike_df = pd.read_csv('/Users/crunchy__child/seoul-bike-demand/data/processed/merged_bike_data.csv')

# Weather data set(Need to preprocess data)
# Column names is in 3rd row
weather_df = pd.read_csv('/Users/crunchy__child/seoul-bike-demand/data/raw/seoul_weather.csv', encoding='cp949', header = 2)

In [3]:
# Change column names to english
weather_df.columns = [
    'station_id', 
    'station_name', 
    'date', 
    'avg_temp_C', 
    'min_temp_C', 
    'max_temp_C', 
    'daily_rain_mm', 
    'avg_wind_mps', 
    'avg_humidity_pct', 
    'total_sunlight_hr'
]

# Check
# weather_df.head()

In [4]:
# Data size (Row, Column)
print(weather_df.shape)

# Check the data type
print(weather_df.info())

# Statistically Summary
print(weather_df.describe())

(92, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   station_id         92 non-null     int64  
 1   station_name       92 non-null     object 
 2   date               92 non-null     object 
 3   avg_temp_C         92 non-null     float64
 4   min_temp_C         92 non-null     float64
 5   max_temp_C         92 non-null     float64
 6   daily_rain_mm      55 non-null     float64
 7   avg_wind_mps       92 non-null     float64
 8   avg_humidity_pct   92 non-null     float64
 9   total_sunlight_hr  92 non-null     float64
dtypes: float64(7), int64(1), object(2)
memory usage: 7.3+ KB
None
       station_id  avg_temp_C  min_temp_C  max_temp_C  daily_rain_mm  \
count        92.0   92.000000   92.000000   92.000000      55.000000   
mean        108.0   25.784783   22.667391   29.622826      17.338182   
std           0.0    2.792845

In [5]:
# Missing value by column
print(weather_df.isnull().sum())

# Missing value rate by column
print(weather_df.isnull().mean())

station_id            0
station_name          0
date                  0
avg_temp_C            0
min_temp_C            0
max_temp_C            0
daily_rain_mm        37
avg_wind_mps          0
avg_humidity_pct      0
total_sunlight_hr     0
dtype: int64
station_id           0.000000
station_name         0.000000
date                 0.000000
avg_temp_C           0.000000
min_temp_C           0.000000
max_temp_C           0.000000
daily_rain_mm        0.402174
avg_wind_mps         0.000000
avg_humidity_pct     0.000000
total_sunlight_hr    0.000000
dtype: float64


In [6]:
# Since daily_rain_mm value for non rainy day is Nan, replaces it to 0
weather_df['daily_rain_mm'] = weather_df['daily_rain_mm'].fillna(0)

# Check
# print(weather_df.isnull().sum())

In [7]:
# 1. Convert to datetime (put errors='coerce' to filter invalid form)
weather_df['date'] = pd.to_datetime(weather_df['date'], errors='coerce')

# Check the number of NaT (Optional, but necessary if the data is large like yearly)
# print(weather_df['date'].isna().sum())

# 2. Drop filtered(NaT) row
weather_df = weather_df.dropna(subset=['date'])

# 3. Filter the range of date (2023-06-01~2023-08-31)
weather_df = weather_df[(weather_df['date'] >= '2023-06-01') & (weather_df['date'] <= '2023-08-31')]

In [8]:
# Create derived variable

# Daily temperature range
weather_df['temp_range'] = weather_df['max_temp_C'] - weather_df['min_temp_C']

# True(1) if it rained, False(0) otherwise
weather_df['rain_binary'] = weather_df['daily_rain_mm'].apply(lambda x: 1 if x > 0 else 0)

In [9]:
# Drop unnecessary columns from merged_bike_data
columns_to_keep = [
    'rental_datetime', 'rental_station_name', 'duration_min', 'distance_m',
    'rental_year', 'rental_month', 'rental_day', 'rental_weekday', 'rental_hour', 'is_weekend'
]

bike_df = bike_df[columns_to_keep]


In [ ]:
# Convert rental_datetime column to datetime and get date from rental_datetime
bike_df['rental_datetime'] = pd.to_datetime(bike_df['rental_datetime'], errors='coerce')
bike_df['date'] = bike_df['rental_datetime'].dt.normalize()

# Convert date column to datetime
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Merge two data set using date column
merged_df = pd.merge(bike_df, weather_df, on='date', how='left')

# Drop unnecessary columns
merged_df = merged_df.drop(columns=['date', 'station_name', 'station_id'])

# Check
# print(merged_df.head())

In [13]:
# Store merged csv file
merged_df.to_csv("/Users/crunchy__child/seoul-bike-demand/data/processed/bike_with_weather.csv", index=False)
